# A new Pipeline Approach:


## TORITOS Pipeline

### Co-workers: J. B. Cabral, M. Beroiz, TOROS Collaboration

## What is a pipeline?

*A pipeline can be understood in a collection of filters and connectors that consume data and perform transformation of it on a linear and repetitive fashion.*

This data that is consumed can be named as a *stream* and each filter is a transformation. Connectors between the filters are the schema of the pipeline, and it inspires a *always forward* structure.

## Why do we do this?

* Observational astronomers are facing a data *tsunami*, and synoptic surveys are one of the more data intensive tasks today.

* Telescopes of any size can produce massive amounts of data. 

* Processing is time consuming for humans, and most of the tasks involved are likely to fail.

* We need this for TOROS/TORITOS